In [1]:
# import timbermafia
import logging
import re
import sys
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
def test_log_with_a_very_long_name():
    log.debug('test from func')
replacer_flag = 'timbermafia_replacer_f0df7d32d480'

# class MyClass(timbermafia.Logged):
#     def status(self):
#         self.log.info(f'logging from {self.__class__.__name__} in the function status')

In [90]:
s1 = '__{|} __{c} {asctime} _{|} {level} _{} {name}.{funcName} __{>>} {message}'

# Get a list of all regex matches for anything before a _{} deliminator
myre = '.*?_+{.*?}'
myre_greedy = '.*?_+{.*}'
l1 = re.findall(myre, s1)
l1
found = ''.join(l1)

# My shitty regex means an edge case where 
if found == s1:
    print('all accounted for')
else:
    print('not all accounted for')
    missed = s1.replace(found, '')
    print(missed)
    l1.append(missed)
    
l1


['__{|}',
 ' __{c}',
 ' {asctime} _{|}',
 ' {level} _{}',
 ' {name}.{funcName} __{>>}']

not all accounted for
 {message}


['__{|}',
 ' __{c}',
 ' {asctime} _{|}',
 ' {level} _{}',
 ' {name}.{funcName} __{>>}',
 ' {message}']

In [45]:
s1
myre2 = '_+{.*?}/^[_]*/'
re.findall(myre2, s1)

'__{|} {asctime} _{|} {level} _{|} {name}.{funcName} __{>>} {message}'

[]

In [56]:
re.findall(r'.+[^;]', 'sdgdsagsd ; fsaFSDGF ; wqerwqrweqr')

['sdgdsagsd ; fsaFSDGF ; wqerwqrweqr']

In [84]:
s = 'aaaaa Z bbbbbbb Z ccccccc Z dddddd'
re.findall('.+Z', s)
re.findall('Z.+', s)

['aaaaa Z bbbbbbb Z ccccccc Z']

['Z bbbbbbb Z ccccccc Z dddddd']